# Enumerations

amaranth.lib.enum模块是标准python enum模块的一个可替换版本，它提供了扩展的Enum，IntEnum，Flag和IntFlag

并具备指定shape的能力，可使用shape=4，指定其shape。

In [20]:
%config InteractiveShell.ast_node_interactivity = "all"

from amaranth import *
from amaranth.lib import enum

class Funct(enum.Enum, shape=4):
    ADD = 0
    SUB = 1
    MUL = 2

Shape.cast(Funct)
Value.cast(Funct.ADD)

unsigned(4)

(const 4'd0)

任何constant-castable的常量表达式都可以用作成员的值。

In [21]:
class Op(enum.Enum, shape=1):
    REG = 0
    IMM = 1

class Instr(enum.Enum, shape=5):
    ADD  = Cat(Funct.ADD, Op.REG)  # 0_0000    0
    ADDI = Cat(Funct.ADD, Op.IMM)  # 1_0000    16
    SUB  = Cat(Funct.SUB, Op.REG)  # 0_0001    1
    SUBI = Cat(Funct.SUB, Op.IMM)  # 1_0001    17
    MUL  = Cat(Funct.MUL, Op.REG)  # 0_0010    2

Instr.ADD
Instr.ADDI
Instr.SUB
Instr.SUBI
Instr.MUL

<Instr.ADD: 0>

<Instr.ADDI: 16>

<Instr.SUB: 1>

<Instr.SUBI: 17>

<Instr.MUL: 2>

shape=是可选的，如果没有指定，那么你定义的class是完全继承于Python的enum

In [22]:
import amaranth.lib.enum

class NormalEnum(amaranth.lib.enum.Enum):
    SPAM = 0
    HAM = 1

通过这种方式，这个模块可以作为标准enum模块的直接代替。

在Amaranth项目中，所有的import enum语句都可以替换为 from amaranth.lib import enum

当Signal使用Enum或Flag类型作为参数时，会自动包装在EnumView或FlagView包装器。

任何value-like的内容也可以通过将其强制转化enum

In [23]:
a = Signal(Funct)
b = Signal(Op)
c = Signal(4)

type(a)
type(c)

Funct.ADD == 1            # 与值想比较
Funct.MUL == Funct.SUB    # 与同类型的
Funct.SUB == Op.IMM       # 与其他EnumView进行比较
# a == b EnumView对象只能与相同枚举类型的值或其他EnumView进行比较

amaranth.lib.enum.EnumView

amaranth.hdl._ast.Signal

False

False

False

与标准的Python enum.IntEnum和enum.IntFlag类似，Amaranth的IntEnum和IntFlag是松散类型(loosely typed)

因此不会像不同枚举那样被封装在view class中。

In [24]:
class TransparentEnum(enum.IntEnum, shape=unsigned(4)):
    FOO = 0
    BAR = 1

a = Signal(TransparentEnum)
type(a) is Signal

True

还可以为给定枚举定义一个自定义的view class

In [25]:
class InstrView(enum.EnumView):
    def has_immediate(self):
        return (self == Instr.ADDI) | (self == Instr.SUBI)

class Instr(enum.Enum, shape=5, view_class=InstrView):
    ADD  = Cat(Funct.ADD, Op.REG)
    ADDI = Cat(Funct.ADD, Op.IMM)
    SUB  = Cat(Funct.SUB, Op.REG)
    SUBI = Cat(Funct.SUB, Op.IMM)

a = Signal(Instr)
type(a)
a.has_immediate()

__main__.InstrView

(| (== (sig a) (const 5'd16)) (== (sig a) (const 5'd17)))